In [1]:
!pip install langchain langchain-community OpenAI indoNLP emoji nlpaug Sastrawi sentence-transformers transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [2]:
!python -m spacy download xx_ent_wiki_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 51.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# **Import Modules**

In [112]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import nltk
import shutil
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm.auto as tqdm
import spacy
import json
import os
import torch
import emoji
import random


from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from indoNLP.preprocessing import emoji_to_words, replace_slang, pipeline
from __future__ import print_function
from transformers import BertTokenizerFast, AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel
from transformers import pipeline
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# **Import Data**

In [99]:
df = pd.read_csv("/content/dataset_ppn-12-persen_cnn.csv")

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      103 non-null    object
 1   content    101 non-null    object
 2   link       103 non-null    object
 3   timestamp  103 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


# **Preprocessing Data**

In [101]:
df.dropna(subset=['content'], inplace=True)

In [102]:
def data_cleaning_pipeline(text: str):
    step_1 = re.sub(r'ADVERTISEMENT SCROLL TO CONTINUE WITH CONTENT', '', text, flags=re.IGNORECASE)
    step_2 = re.sub(r'\xa0', ' ', step_1)
    step_3 = re.sub(r'\[Gambas:Video CNN\]', '', step_2)

    return step_3.strip()

In [103]:
df['content'] = df['content'].apply(data_cleaning_pipeline)

# **Feature Engineering**

In [105]:
def feature_engineering(text):
    char_count = len(text)
    word_count = len(text.split())
    sentence_count = len(text.split("."))
    average_word_length = round(sum(len(word) for word in text.split()) / word_count)
    token_count = round(len(text) / 6)

    return char_count, word_count, sentence_count, average_word_length, token_count

In [106]:
df['char_count'], df['word_count'], df['sentence_count'], df['average_word_length'], df['token_count'] = zip(*df['content'].apply(feature_engineering))

In [107]:
nlp = spacy.load("xx_ent_wiki_sm")
nlp.add_pipe('sentencizer')

def split_sentence_using_spacy(text):
    doc = nlp(text)
    data = []
    for sent in doc.sents:

        if len(sent.text) != 0:
            data.append(sent.text)

    return data

In [108]:
df['sentences'] = df['content'].apply(split_sentence_using_spacy)

In [109]:
df.describe().round(1)

,char_count,word_count,sentence_count,average_word_length,token_count
count,101.0,101.0,101.0,101.0,101.0
mean,2294.8,319.5,21.9,6.1,382.4
std,1442.0,194.3,13.5,0.4,240.3
min,196.0,29.0,4.0,5.0,33.0
25%,1559.0,224.0,15.0,6.0,260.0
50%,2095.0,289.0,19.0,6.0,349.0
75%,2571.0,365.0,25.0,6.0,428.0
max,10625.0,1440.0,86.0,7.0,1771.0


# **Split sentence into small chunks**

In [114]:
num_sentence_chunk_size = 10

def split_list(input_list: list[str],
               slice_size: int=num_sentence_chunk_size) -> list[list[str]]:
    return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]

def chunks(text):
    sentence_chunks = split_list(input_list=text, slice_size=num_sentence_chunk_size)
    num_chunks = len(sentence_chunks)

    return sentence_chunks, num_chunks

df['sentence_chunks'], df['num_chunks'] = zip(*df['sentences'].apply(chunks))

In [128]:
df.describe().round(2)

,char_count,word_count,sentence_count,average_word_length,token_count,num_chunks
count,101.00,101.00,101.00,101.00,101.00,101.00
mean,2294.82,319.50,21.87,6.11,382.45,2.34
std,1442.02,194.34,13.54,0.37,240.31,1.24
min,196.00,29.00,4.00,5.00,33.00,1.00
25%,1559.00,224.00,15.00,6.00,260.00,2.00
50%,2095.00,289.00,19.00,6.00,349.00,2.00
75%,2571.00,365.00,25.00,6.00,428.00,3.00
max,10625.00,1440.00,86.00,7.00,1771.00,9.00
